<a href="https://colab.research.google.com/github/ilyanovak/Quality_Of_Government/blob/master/quality_of_government.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**TODO** 
1. Confirm no gov countries removed
2. Finish linear imputation
3. Do mean imputation
4. Correlations matrix of gov variables and average correlation
5. Finish PCA
6. Plot scree graph
7. Move gov to the bottom
8. Review all dataset names one more time
9. Review all descriptions
10. Perform multiple regression with and without stanardization and logarithms and encoding to get R^2
11. Do feature importance and plot it
12. Do plotly dash graph correlations 
13. XGBoost

# Import libraries

In [323]:
!pip install xlrd

import pandas as pd
import numpy as np

import plotly.express as px

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [324]:
# def enable_plotly_in_cell():
#   import IPython
#   from plotly.offline import init_notebook_mode
#   display(IPython.core.display.HTML('''<script src="/static/components/requirejs/require.js"></script>'''))
#   init_notebook_mode(connected=False)

# Create global variables

In [325]:
# Data path to project's github repository

DATA_PATH = 'https://raw.githubusercontent.com/ilyanovak/Lambda_Project_1/master/data/'

In [326]:
'''
This is a list of country names to be dropped from datasets for one of several reasons:
(1) They refer to geopolitic blocks rather than distinct countries
(2) They are tiny nations and therefore outliers
(3) There is no data for them within the relevant time period; 1996-2018
(4) They are not included in the quality of government dataset
(5) There is so little data available that they are not worth it
'''

country_drop_list = [
                'Arab World', 'Central Europe and the Baltics',
                'Caribbean small states', 'East Asia & Pacific (excluding high income)'
                'Early-demographic dividend', 'East Asia & Pacific',
                'East Asia & Pacific (excluding high income)', 'Early-demographic dividend', 
                'Europe & Central Asia (excluding high income)',
                'Europe & Central Asia', 'Euro area', 'European Union', 
                'Fragile and conflict affected situations', 'High income',
                'Heavily indebted poor countries (HIPC)', 'IBRD only',
                'IDA & IBRD total', 'IDA total', 'IDA blend', 'IDA only',
                'Not classified', 'Latin America & Caribbean (excluding high income)',
                'Latin America & Caribbean', 'Least developed countries: UN classification', 
                'Low income', 'Lower middle income',  'Low & middle income',
                'Late-demographic dividend', 'Middle East & North Africa',
                'Middle income', 'Middle East & North Africa (excluding high income)', 
                'North America', 'OECD members', 'Other small states', 
                'Pre-demographic dividend', 'Pacific island small states', 
                'Post-demographic dividend', 'South Asia', 
                'Sub-Saharan Africa (excluding high income)', 'Sub-Saharan Africa', 
                'Small states', 'East Asia & Pacific (IDA & IBRD countries)',
                'Europe & Central Asia (IDA & IBRD countries)', 
                'Latin America & the Caribbean (IDA & IBRD countries)', 
                'Middle East & North Africa (IDA & IBRD countries)', 
                'South Asia (IDA & IBRD)', 'Sub-Saharan Africa (IDA & IBRD countries)', 
                'Upper middle income', 'World', 'Yugoslavia', 'USSR', 
                'German Democratic Republic', 'Republic of Vietnam', 'Yemen PDR', 
                'Czechoslovakia', 'Soviet Union','Cook Islands', 'Niue', 
                'Martinique', 'Netherlands Antilles (former)', 'Curacao', 
                'Faroe Islands', 'French Polynesia', 'Gibraltar', 'Isle of Man', 
                'Netherlands Antilles (former)', 'New Caledonia', 
                'Northern Mariana Islands', 'Sint Maarten (Dutch part)', 
                'St. Martin (French part)','Turks and Caicos Islands', 
                'French Guiana', 'Réunion', 'Africa', 'Asia', 'Caribbean', 
                'Central America', 'Central Asia', 'Eastern Africa', 'Eastern Asia', 
                'Eastern Europe', 'Europe', 'Least developed countries', 
                'Less developed regions', 'Less developed regions, excluding China', 
                'Less developed regions, excluding least developed countries', 
                'Middle Africa', 'More developed regions', 'North Korea', 
                'Northern Africa', 'Northern America', 'Northern Europe', 'Oceania', 
                'South America',  'South-Central Asia', 'South-Eastern Asia', 
                'Southern Africa', 'Southern Asia', 'Southern Europe', 'Western Africa', 
                'Western Asia', 'Western Europe', 'Western Sahara', 'Guadeloupe', 
                'Mayotte', 'Melanesia', 'Micronesia (region)', 'Polynesia', 
                'Reunion', 'Antarctica', 'Bouvet Island', 'British Indian Ocean Territory', 
                'Christmas Island', 'Cocos [Keeling] Islands', 
                'U.S. Minor Outlying Islands', 'Vatican City', 'Wallis and Futuna', 
                'Svalbard and Jan Mayen', 'Saint Helena', 'Gaza Strip', 
                'Netherlands Antilles', 'Falkland Islands [Islas Malvinas]', 
                'French Southern Territories', 'Guernsey', 'Norfolk Island', 
                'Heard Island and McDonald Islands', 'Montserrat', 'Pitcairn Islands', 
                'Saint Pierre and Miquelon', 'South Georgia and the South Sandwich Islands', 
                'Tokelau', 'Developing States', 'Very high human development', 
                'Regions', 'Human Development', 'Arab States','Developing Countries',
                'East Asia and the Pacific', 'Europe and Central Asia', 
                'High human development', 'Latin America and the Caribbean', 
                'Least Developed Countries', 'Low human development', 'Small Island'
                'Medium human development', 'Organization for Economic Co-operation and Development', 
                'Medium human development', 'Small Island Developing States', 
                'Saint Helena, Ascension, and Tristan da Cunha', 'Mariana Islands', 
                'Bosnia', 'Anguilla']

In [327]:
# This dictionary is used to map country name values across datasets in order to harmonize all observations.

country_replace_dict = {
                'Bahamas':'Bahamas, The', 'United States of America':'United States', 
                'Micronesia':'Micronesia, Fed. Sts.', 'Yemen Arab Republic':'Yemen', 
                'Hong Kong SAR, China':'Hong Kong', 'Macedonia, FYR':'North Macedonia',
                'São Tomé and Príncipe':'Sao Tome and Principe', 'Korea, Rep.':'Korea, South',
                'Congo-Kinshasa':'Congo, Dem. Rep.', 'Republic of Korea':'Korea, South',
                'Congo-Brazzaville':'Congo, Rep.','Macedonia':'North Macedonia',
                'Brunei':'Brunei Darussalam','Cape Verde':'Cabo Verde', 
                'Russian Federation':'Russia','Gambia, The':'Gambia',
                'Yemen, Rep.':'Yemen',"Côte d'Ivoire":"Cote d'Ivoire",
                'St. Vincent and Grenadines':'St. Vincent and the Grenadines',
                'Egypt, Arab Rep.':'Egypt','Kyrgyz Republic':'Kyrgyzstan',
                'Venezuela, RB':'Venezuela','Slovakia':'Slovak Republic',
                'Lao PDR':'Laos', 'Syrian Arab Republic':'Syria', 
                'Korea':'Korea, South', 'German Federal Republic':'Germany', 
                'Democratic Republic of Congo':'Congo, Dem. Rep.', 
                'Bosnia-Herzegovina':'Bosnia and Herzegovina',
                'Bosnian Federation':'Bosnia and Herzegovina', 
                'Democratic Republic of Vietnam':'Vietnam', 'Congo':'Congo, Rep.',
                'Timor-Leste':'East Timor', 'Swaziland':'Eswatini', 
                "Democratic People's Republic of Korea":'Korea, North', 
                "Korea, Dem. People’s Rep.":'Korea, North', 'Korea, Rep.':'Korea, South',
                'Macao SAR, China':'Macau', 'Czechia':'Czech Republic',
                'Saint Vincent and the Grenadines':'St. Vincent and the Grenadines',
                'Korea, North':'Korea, North', 'Korea, Dem. People’s Rep.':'Korea, North', 
                'Saint Lucia':'St. Lucia', 'Iran, Islamic Rep.':'Iran',
                'Congo, Republic of the':'Congo, Rep.', 'eSwatini':'Eswatini',
                'Central African Rep.':'Central African Republic',
                'Saint Kitts and Nevis':'St. Kitts and Nevis',
                'Micronesia, Federated States of':'Micronesia, Fed. Sts.',
                'Virgin Islands':'British Virgin Islands', 'Yemen, Rep.':'Yemen',
                'Taiwan, China':'Taiwan', 'Korea, Dem. Rep.':'Korea, North',
                'São Tomé and Principe':'Sao Tome and Principe',
                'Jersey, Channel Islands':'Channel Islands', 'Macao':'Macau', 
                'Palestinian Territories':'West Bank and Gaza', 
                'South Korea':'Korea, South', 'Timor':'East Timor', 
                'Micronesia (country)':'Micronesia, Fed. Sts.', 
                'Palestine':'West Bank and Gaza', 'Australia/New Zealand':'Australia',
                'United States Virgin Islands':'Virgin Islands (U.S.)', 
                'U.S. Virgin Islands':'Virgin Islands (U.S.)', 
                'Macedonia [FYROM]':'North Macedonia', 'Jersey':'Channel Islands', 
                'Congo [DRC]':'Congo, Dem. Rep.', 'Congo [Republic]':'Congo, Rep.', 
                'Myanmar [Burma]':'Myanmar', 'Viet Nam':'Vietnam', 
                'Eswatini (Kingdom of)':'Eswatini', 'Moldova (Republic of)':'Moldova',
                'Palestine, State of':'West Bank and Gaza', 
                'Hong Kong, China (SAR)':'Hong Kong', 'Iran (Islamic Republic of)':'Iran', 
                'Korea (Republic of)':'Korea, South', "Lao People's Democratic Republic":"Laos", 
                'Venezuela (Bolivarian Republic of)':'Venezuela', 
                'Congo (Democratic Republic of the)':'Congo, Dem. Rep.', 
                'Micronesia (Federated States of)':'Micronesia, Fed. Sts.', 
                'Bolivia (Plurinational State of)':'Bolivia', 
                'Tanzania (United Republic of)':'Tanzania', 
                'Congo, Democratic Republic of the':'Congo, Dem. Rep.', 
                'Great Britain': 'United Kingdom', 'Dominican Rep.':'Dominican Republic', 
                'Czech Rep.':'Czech Republic', 'Cyprus (G)':'Cyprus', 
                'Serbia and Montenegro':'Serbia', 'Hong Kong, China':'Hong Kong',
                'Macao, China':'Macao'}

In [328]:
# This list of years is used because only the time period between 1996-2018 
# is available in the quality of government dataset. 

year_list = list(range(1996, 2002, 2)) + list(range(2002, 2019, 1))

# Import and clearn target dataset

In [329]:
'''
Title: Quality of Government
Source: The Worldwide Governance Indicators Project
Location: http://info.worldbank.org/governance/wgi/
Description: "Governance consists of the traditions and institutions by which 
authority in a country is exercised." ... "The aggregate indicators are based on 
several hundred individual underlying variables, taken from a wide variety of 
existing data sources. The data reflect the views on governance of survey 
respondents and public, private, and NGO sector experts worldwide."
'''

gov_excel = pd.read_excel(DATA_PATH+'wgidataset.xlsx', 
                       header=[13,14], 
                       sheet_name=['VoiceandAccountability',
                                   'Political StabilityNoViolence',
                                   'GovernmentEffectiveness',
                                   'RegulatoryQuality',
                                   'RuleofLaw',
                                   'ControlofCorruption'])

gov = pd.DataFrame(columns=['Country Name', 
                                   'Year', 
                                   'Voice and Accountability', 
                                   'Political Stability No Violence',
                                   'Government Effectiveness',
                                   'Regulatory Quality',
                                   'Rule of Law',
                                   'Control of Corruption'])

# Inserts data from multiple excel sheets into a single data frame
for i in range(0, len(gov_excel['VoiceandAccountability'])):
  for year in year_list:
    gov = gov.append({'Country Name':gov_excel['VoiceandAccountability'][('Unnamed: 0_level_0','Country/Territory')][i], 
                    'Year':year,
                    'Voice and Accountability':gov_excel['VoiceandAccountability'][(year,'Estimate')][i], 
                    'Political Stability No Violence':gov_excel['Political StabilityNoViolence'][(year,'Estimate')][i], 
                    'Government Effectiveness':gov_excel['GovernmentEffectiveness'][(year,'Estimate')][i], 
                    'Regulatory Quality':gov_excel['RegulatoryQuality'][(year,'Estimate')][i], 
                    'Rule of Law':gov_excel['RuleofLaw'][(year,'Estimate')][i], 
                    'Control of Corruption':gov_excel['ControlofCorruption'][(year,'Estimate')][i]}, 
                    ignore_index=True)
    
gov['Year'] = gov['Year'].astype(int)
# gov = gov[~gov['Country Name'].isin(country_drop_list)]
gov['Country Name'] = gov['Country Name'].replace(country_replace_dict)

# Import and wrangle independent variables

In [330]:
'''
Title: Population, total
Source: World Bank
Location: https://data.worldbank.org/indicator/SP.POP.TOTL
Description: Total population is based on the de facto definition of population, which counts all residents regardless of legal status or citizenship. The values shown are midyear estimates.
'''

pop = pd.read_csv(DATA_PATH+'pop.csv', header=2)
pop = pop.drop(['Country Code', 
                'Indicator Name', 
                'Indicator Code', 
                'Unnamed: 64'], axis=1)

pop = pd.melt(pop, id_vars=['Country Name'], var_name='Year', value_name='Total Population')

pop['Year'] = pop['Year'].astype(int)
pop = pop[~pop['Country Name'].isin(country_drop_list)]
pop['Country Name'] = pop['Country Name'].replace(country_replace_dict)

In [331]:
'''
Title: GDP per capita (current US$)
Source: World Bank
Location: https://data.worldbank.org/indicator/NY.GDP.PCAP.CD
Description: GDP per capita is gross domestic product divided by midyear 
population. Data are in current U.S. dollars."
'''

gdp = pd.read_csv(DATA_PATH+'gdp.csv', header=2)
gdp = gdp.drop(['Country Code', 
                'Indicator Name', 
                'Indicator Code', 
                'Unnamed: 64'], axis=1)

gdp = pd.melt(gdp, id_vars=['Country Name'], var_name='Year', value_name='GDP per Capita')

gdp['Year'] = gdp['Year'].astype(int)
gdp = gdp[~gdp['Country Name'].isin(country_drop_list)]
gdp['Country Name'] = gdp['Country Name'].replace(country_replace_dict)

In [332]:
'''
Title: Gini Coefficient
Source: The Standardized World Income Inequality Database.
Location: https://fsolt.org/swiid/
Description: Gini index of "inequality in market (pre-tax, pre-transfer) income
'''

gini = pd.read_csv(DATA_PATH+'gini.csv', header=0)
gini = gini[['country', 'year', 'gini_disp']]
gini.columns = ['Country Name', 'Year', 'Gini Coefficient']
gini = gini[~gini['Country Name'].isin(country_drop_list)]
gini['Country Name'] = gini['Country Name'].replace(country_replace_dict)

In [333]:
'''
Title: Ethnic Fractionalization Index
Source: Historical Index of Ethnic Fractionalization Dataset
Location: https://dataverse.harvard.edu/dataset.xhtml?persistentId=doi:10.7910/DVN/4JQRCL
Description: The index "ranges from 0, when there is no ethnic fractionalization 
and all individuals are members of the same ethnic group to 1, where each 
individual belongs to his or her own ethnic group."
'''

hief = pd.read_csv(DATA_PATH+'hief.csv', header=0)
hief.columns = ['Country Name', 'Year', 'Ethnic Fractionalization Index']
hief = hief[~hief['Country Name'].isin(country_drop_list)]
hief['Country Name'] = hief['Country Name'].replace(country_replace_dict)

In [334]:
'''
Title: Human Capital Index
Source: Human Capital Project
Location: https://www.worldbank.org/en/publication/human-capital
Description: The HCI calculates the contributions of health and education to 
worker productivity. The final index score ranges from zero to one, and measures 
the productivity as a future worker of child born todat relative to the benchmark 
of full health and complete education."
'''

hc_initial = pd.read_excel(DATA_PATH+'humancapital.xlsx', 
                         header=[0], sheet_name=['HCI - MaleFemale'])
hc_initial = hc_initial['HCI - MaleFemale'][['Country Name', 'HUMAN CAPITAL INDEX']]
hc_initial.columns = ['Country Name', 'Human Capital Index']

'''
Imported data is exclusively from 2018. I assume the human capital values in that year
are a good approximation for 1996-2018. I perform data data imputation by inserting 
each row into an empty data frame 20 times, each time using a different year in year_list 
'''
hc = pd.DataFrame(columns=['Country Name', 'Year', 'Human Capital Index'])
j = 0
for i in range(0, hc_initial.shape[0]):
  for year in year_list:
    hc.loc[j] = [hc_initial['Country Name'][i], year, hc_initial['Human Capital Index'][i]]
    j += 1
    
hc = hc[~hc['Country Name'].isin(country_drop_list)]
hc['Country Name'] = hc['Country Name'].replace(country_replace_dict)

In [335]:
'''
Title: Intentional homicides (per 100,000 people)
Source: World Bank
Location: https://data.worldbank.org/indicator/VC.IHR.PSRC.P5 
Description: Intentional homicides are estimates of unlawful homicides purposely 
inflicted as a result of domestic disputes, interpersonal violence, violent 
conflicts over land resources, intergang violence over turf or control, and 
predatory violence and killing by armed groups.
'''

homicide = pd.read_csv(DATA_PATH+'homicide.csv', header=2)
homicide = homicide.drop(['Country Code', 
                'Indicator Name', 
                'Indicator Code', 
                'Unnamed: 64'], axis=1)

homicide = pd.melt(homicide, id_vars=['Country Name'], var_name='Year', value_name='Homicide Rate')

homicide['Year'] = homicide['Year'].astype(int)
homicide = homicide[~homicide['Country Name'].isin(country_drop_list)]
homicide['Country Name'] = homicide['Country Name'].replace(country_replace_dict)

In [336]:
'''
Title: Life expectancy at birth, total (years)
Source: World Bank
Location: https://data.worldbank.org/indicator/SP.DYN.LE00.IN
Description: "Life expectancy at birth indicates the number of years a newborn 
infant would live if prevailing patterns of mortality at the time of its birth 
were to stay the same throughout its life."
'''

life = pd.read_csv(DATA_PATH+'lifeexpectancy.csv', header=2)
life = life.drop(['Country Code', 
                'Indicator Name', 
                'Indicator Code',
                'Unnamed: 64'], axis=1)

life = pd.melt(life, id_vars=['Country Name'], var_name='Year', value_name='Life Expectancy')

life['Year'] = life['Year'].astype(int)
life = life[~life['Country Name'].isin(country_drop_list)]
life['Country Name'] = life['Country Name'].replace(country_replace_dict)

In [337]:
'''
Title: Fertility rate, total (births per woman)
Source: World Bank
Location: https://data.worldbank.org/indicator/SP.DYN.TFRT.IN
Description: Total fertility rate represents the number of children that would 
be born to a woman if she were to live to the end of her childbearing years and 
bear children in accordance with age-specific fertility rates of the specified year.
'''

fertility = pd.read_csv(DATA_PATH+'fertility.csv', header=2)
fertility = fertility.drop(['Country Code', 
                'Indicator Name', 
                'Indicator Code',
                'Unnamed: 64'], axis=1)

fertility = pd.melt(fertility, id_vars=['Country Name'], var_name='Year', value_name='Fertility Rate')

fertility['Year'] = fertility['Year'].astype(int)
fertility = fertility[~fertility['Country Name'].isin(country_drop_list)]
fertility['Country Name'] = fertility['Country Name'].replace(country_replace_dict)

In [338]:
'''
Title: National IQ Score
Source: The National IQ Dataset
Location: https://viewoniq.org/
Description: National IQ-scores from Lynn & Vanhanen (2012) estimated 
from psychometric tests only.
'''

iq_excel = pd.read_excel(DATA_PATH+'intelligence.xlsx', 
                         header=[1], sheet_name=['FAV'], skipfooter=7, usecols=['Unnamed: 1','L&V12'])
iq_initial = iq_excel['FAV'][['Unnamed: 1','L&V12']]
iq_initial.columns = ['Country Name','National IQ Score']

'''
Imported data is exclusively from 2012. I assume the IQ values in that year
are a good approximation for 1996-2018. I perform data data imputation by inserting 
each row into an empty data frame 20 times, each time using a different year in year_list 
'''
iq = pd.DataFrame(columns=['Country Name', 'Year', 'National IQ Score'])
j = 0
for i in range(0, iq_initial.shape[0]):
  for year in year_list:
    iq.loc[j] = [iq_initial['Country Name'][i], year, iq_initial['National IQ Score'][i]]
    j += 1

iq = iq[~iq['Country Name'].isin(country_drop_list)]
iq['Country Name'] = iq['Country Name'].replace(country_replace_dict)

In [339]:
'''
Title: Country Distance from Equator
Source: Google
Location: https://developers.google.com/public-data/docs/canonical/countries_csv
Description: Country distance from equator calculated from country latitude
'''

equator_initial = pd.read_csv(DATA_PATH+'equator.csv', header=0)

# Distance from Equator is calculated as absolute value of Latitude
equator_initial['Distance from Equator'] = equator_initial['latitude'].abs()

# Data imputation: A country's distance from the equator is constant across years
# Inserts each row into an empty data frame 20 times, each time using a different year in year_list 
equator = pd.DataFrame(columns=['Country Name', 'Year', 'Distance from Equator'])
j = 0
for i in range(0, equator_initial.shape[0]):
  for year in year_list:
    equator.loc[j] = [equator_initial['name'][i], year, equator_initial['Distance from Equator'][i]]
    j += 1

equator['Year'] = equator['Year'].astype(int)
equator = equator[~equator['Country Name'].isin(country_drop_list)]
equator['Country Name'] = equator['Country Name'].replace(country_replace_dict)

In [340]:
'''
Title: Trade (% of GDP)
Source: World Bank
Location: https://data.worldbank.org/indicator/NE.TRD.GNFS.ZS
Description: "Trade (% of GDP), Trade is the sum of exports and 
imports of goods and services measured as a share of gross domestic product."
'''

trade = pd.read_csv(DATA_PATH+'trade.csv', header=2)
trade = trade.drop(['Country Code', 
                'Indicator Name', 
                'Indicator Code', 
                'Unnamed: 64'], axis=1)

trade = pd.melt(trade, id_vars=['Country Name'], var_name='Year', value_name='Trade (% of GDP)')

trade['Year'] = trade['Year'].astype(int)
trade = trade[~trade['Country Name'].isin(country_drop_list)]
trade['Country Name'] = trade['Country Name'].replace(country_replace_dict)

In [341]:
'''
Title: Gender Development Index (GDI)
Source: United Nations Development Programme
Location: http://hdr.undp.org/en/content/gender-development-index-gdi
Description: "The GDI measures gender gaps in human development achievements by 
accounting for disparities between women and men in three basic dimensions of 
human development—health, knowledge and living standards using the same component 
indicators as in the HDI. The GDI is the ratio of the HDIs calculated separately 
for females and males using the same methodology as in the HDI. It is a direct 
measure of gender gap showing the female HDI as a percentage of the male HDI."
'''

gender = pd.read_csv(DATA_PATH+'gender.csv', na_values='..', skiprows = 0, header=1, 
                     usecols=['Country', '2000', '2005', '2010', '2011', '2012', 
                              '2013', '2014', '2015', '2016', '2017', '2018'])

gender = pd.melt(gender, id_vars=['Country'], var_name='Year', value_name='Gender Development Index')

gender.columns = ['Country Name', 'Year', 'Gender Development Index']
gender['Year'] = gender['Year'].astype(int)
gender = gender[~gender['Country Name'].isin(country_drop_list)]
gender['Country Name'] = gender['Country Name'].replace(country_replace_dict)

In [342]:
'''
Title: Median Age
Source: Our World in Data
Location: https://ourworldindata.org/grapher/median-age
Description: Median Age in Years
'''

age = pd.read_csv(DATA_PATH+'age.csv', header=0)
age = age.drop(['Code'], axis=1)

age.columns = ['Country Name', 'Year', 'Median Age']
age = age[~age['Country Name'].isin(country_drop_list)]
age['Country Name'] = age['Country Name'].replace(country_replace_dict)

In [343]:
'''
Title: General government final consumption expenditure (% of GDP)
Source: World Bank
Location: https://data.worldbank.org/indicator/NE.CON.GOVT.ZS
Description: 
General government final consumption expenditure (formerly general government 
consumption) includes all government current expenditures for purchases of goods 
and services (including compensation of employees). It also includes most 
expenditures on national defense and security, but excludes government military 
expenditures that are part of government capital formation.
'''

expenditure = pd.read_csv(DATA_PATH+'expenditure.csv', header=2)
expenditure = expenditure.drop(['Country Code',
                                'Indicator Name',
                                'Indicator Code',
                                'Unnamed: 64'], axis=1)

expenditure = pd.melt(expenditure, id_vars=['Country Name'], var_name='Year', value_name='Public Expenditure per Capita')

expenditure['Year'] = expenditure['Year'].astype(int)
expenditure = expenditure[~expenditure['Country Name'].isin(country_drop_list)]
expenditure['Country Name'] = expenditure['Country Name'].replace(country_replace_dict)

In [344]:
'''
Title: Unemployment, total (% of total labor force)
Source: World Bank
Location: https://data.worldbank.org/indicator/SL.UEM.TOTL.ZS
Description: 
Unemployment refers to the share of the labor force that is without work 
but available for and seeking employment.
'''

unemployment = pd.read_csv(DATA_PATH+'unemployment.csv', header=2)
unemployment = unemployment.drop(['Country Code',
                                'Indicator Name',
                                'Indicator Code',
                                'Unnamed: 64'], axis=1)

unemployment = pd.melt(unemployment, id_vars=['Country Name'], var_name='Year', value_name='Unemployment Rate')

unemployment['Year'] = unemployment['Year'].astype(int)
unemployment = unemployment[~unemployment['Country Name'].isin(country_drop_list)]
unemployment['Country Name'] = unemployment['Country Name'].replace(country_replace_dict)

In [345]:
'''
World values survey: Importance in Life: Religion
Source: World Values Survey
Location: http://www.worldvaluessurvey.org/wvs.jsp via Kaggle: https://www.kaggle.com/fernandol/world-values-survey
Description: 
The WVS consists of nationally representative surveys conducted in almost 100 
countries which contain almost 90 percent of the world’s population, using a 
common questionnaire. The WVS is the largest non-commercial, cross-national, 
time series investigation of human beliefs and values ever executed, currently 
including interviews with almost 400,000 respondents. 
'''

values = pd.read_csv(DATA_PATH+'values_data.csv',
                          header=0,
                          na_values='NA',
                          usecols=['Country', 'S020', 'A006'])

values.columns = ['Country Name', 'Year', 'Important in life: Religion']
values['Year'] = values['Year'].astype(int)
values = values[~values['Country Name'].isin(country_drop_list)]
values['Country Name'] = values['Country Name'].replace(country_replace_dict)

In [346]:
'''
Title: PCT patents, applications/million pop.
Source: World Bank
Location: https://tcdata360.worldbank.org/indicators/entrp.pct?country=BRA&indicator=3441&viz=line_chart&years=2012,2016
Description: Number of applications filed under the Patent Cooperation Treaty (PCT) per million population
'''

patents = pd.read_csv(DATA_PATH+'patents.csv')

mask = (patents['Indicator'] == 'PCT patents, applications/million pop.') & (patents['Subindicator Type'] == 'per million pop.')
patents = patents[mask]

patents = patents.drop(columns=['Country ISO3', 'Indicator','Indicator Id', 'Subindicator Type'])

patents = pd.melt(patents, id_vars=['Country Name'], var_name='Year', value_name='Patents Rate')

patents['Year'] = patents['Year'].astype(int)
patents = patents[~patents['Country Name'].isin(country_drop_list)]
patents['Country Name'] = patents['Country Name'].replace(country_replace_dict)

In [347]:
'''
Title: Human Development Index (HDI)
Source: United Nations Development Programme
Location: http://hdr.undp.org/en/content/human-development-index-hdi
Description: 
'''

HDI = pd.read_csv(DATA_PATH+'HDI.csv',
                  skiprows=0,
                  header=1,
                  na_filter=True)

HDI = HDI.rename(columns={'Country':'Country Name'})
HDI = HDI.drop(columns='HDI Rank (2018)')

# Drop columns
i=3
for i in range(3, 60, 2):
  column = 'Unnamed: ' + str(i)
  HDI = HDI.drop(columns=column)

HDI = pd.melt(HDI, id_vars=['Country Name'], var_name='Year', value_name='Human Development Index')

HDI['Year'] = HDI['Year'].astype(int)
HDI = HDI[~HDI['Country Name'].isin(country_drop_list)]
HDI['Country Name'] = HDI['Country Name'].replace(country_replace_dict)

# Replace nan files
HDI['Country Name'] = HDI['Country Name'].replace('nan', np.NaN)

In [348]:
'''
Title: Legal Tradition
Source: Harvard Univesity
Location: https://scholar.harvard.edu/shleifer/publications
Decription: 
'''

legal_initial = pd.read_excel(DATA_PATH+'legal.xls',
                      sheet_name='Table 1',
                      usecols=['country', 
                               'legor_uk', 
                               'legor_fr', 
                               'legor_ge', 
                               'legor_sc',
                               'legor_so'],
                      skipfooter=17)

# Reverse dataset's original one hot encoding
legal_initial['Legal Tradition'] = (legal_initial.iloc[:, 1:] == 1).idxmax(1)

# Remove initial one hot coding
legal_initial = legal_initial.drop(columns=['legor_uk',
                                            'legor_fr', 
                                            'legor_ge', 
                                            'legor_sc', 
                                            'legor_so'])

'''
Data Imputation: 
Imported data is not for a given year. I assume a country's legal tradition is constant across years.
I perform data data imputation by inserting each row into an empty data frame 20 times, each time using a different year in year_list 
'''
legal = pd.DataFrame(columns=['Country Name', 'Year', 'Legal Tradition'])
j = 0
for i in range(0, legal_initial.shape[0]):
  for year in year_list:
    legal.loc[j] = [legal_initial['country'][i], year, legal_initial['Legal Tradition'][i]]
    j += 1

legal['Year'] = legal['Year'].astype(int)
legal = legal[~legal['Country Name'].isin(country_drop_list)]
legal['Country Name'] = legal['Country Name'].replace(country_replace_dict)

# Merge datasets

In [349]:
dataset_list = [pop, gdp, gini, hief, hc, homicide, life, fertility, 
                iq, urban, inflation, equator, trade, gender, age,
                expenditure, unemployment, values, patents, HDI, legal, gov]

df_final = pd.DataFrame(columns=['Country Name', 'Year'])

for dataset in dataset_list:
  df_final = pd.merge(df_final, dataset, on=['Country Name', 'Year'], how='outer')
  print(df_final.shape)

(12420, 3)
(12420, 4)
(12475, 5)
(13640, 6)
(13640, 7)
(13640, 8)
(13640, 9)
(13640, 10)
(13640, 11)
(13640, 12)
(13640, 13)
(13640, 14)
(13640, 15)
(13640, 16)
(17149, 17)
(17149, 18)
(17149, 19)
(17149, 20)
(17149, 21)
(17178, 22)
(17198, 23)
(17338, 29)


# Data Interpolation

In [282]:
for country in df_final['Country Name'].values:
    for column in ['GDP per Capita']:
        index_values = df_final[df_final['Country Name'] == country][column].index
        interpolate_values = df_final[df_final['Country Name'] == country][column].interpolate(method='linear')
        df_final.loc[index_values, column] = interpolate_values

In [308]:
df_final[df_final['Voice and Accountability'].isna() == True]

,Country Name,Year,Total Population,GDP per Capita,Gini Coefficient,Ethnic Fractionalization Index,Human Capital Index,Homicide Rate,Life Expectancy,Fertility Rate,...,Important in life: Religion,Patents Rate,Human Development Index,Legal Tradition,Voice and Accountability,Political Stability No Violence,Government Effectiveness,Regulatory Quality,Rule of Law,Control of Corruption
7475,Aruba,1996.0,83200.0,16586.068436,NaN,NaN,NaN,NaN,73.646000,2.021,...,NaN,NaN,NaN,legor_fr,NaN,NaN,NaN,NaN,NaN,NaN
7483,American Samoa,1996.0,54211.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7509,Channel Islands,1996.0,144335.0,NaN,NaN,NaN,NaN,NaN,77.616000,1.573,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7550,Greenland,1996.0,55900.0,21422.357543,NaN,NaN,NaN,5.4,65.767317,2.493,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7552,Guam,1996.0,148060.0,NaN,NaN,NaN,NaN,NaN,73.589000,2.870,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17193,Macao,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,legor_fr,NaN,NaN,NaN,NaN,NaN,NaN
17194,Macao,2015.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,legor_fr,NaN,NaN,NaN,NaN,NaN,NaN
17195,Macao,2016.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,legor_fr,NaN,NaN,NaN,NaN,NaN,NaN
17196,Macao,2017.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,legor_fr,NaN,NaN,NaN,NaN,NaN,NaN


In [295]:
df_final[df_final['Country Name'] == 'Zimbabwe']

,Country Name,Year,Total Population,GDP per Capita,Gini Coefficient,Ethnic Fractionalization Index,Human Capital Index,Homicide Rate,Life Expectancy,Fertility Rate,...,Important in life: Religion,Patents Rate,Human Development Index,Legal Tradition,Voice and Accountability,Political Stability No Violence,Government Effectiveness,Regulatory Quality,Rule of Law,Control of Corruption
7681,Zimbabwe,1996.0,11541217.0,741.095727,47.7,0.397,0.441194,NaN,49.009,3.989,...,NaN,NaN,0.471,legor_uk,-0.611550,-0.464483,-0.318771,-0.765806,-0.814238,-0.278847
8095,Zimbabwe,1998.0,11747072.0,544.984163,47.7,0.399,0.441194,NaN,46.466,3.839,...,NaN,NaN,0.461,legor_uk,-0.789812,-0.739625,-0.334312,-0.702424,-0.656564,-0.480248
8517,Zimbabwe,2000.0,11881477.0,563.057741,47.6,0.402,0.441194,NaN,44.649,3.748,...,NaN,NaN,0.452,legor_uk,-1.085388,-1.322906,-0.799372,-1.416606,-1.340581,-0.982141
8931,Zimbabwe,2002.0,11954290.0,530.530579,47.5,0.404,0.441194,13.0,43.523,3.718,...,NaN,NaN,0.444,legor_uk,-1.363361,-1.521978,-0.908140,-1.933147,-1.592860,-1.228440
9138,Zimbabwe,2003.0,11982224.0,478.007405,47.4,0.405,0.441194,10.8,43.195,3.725,...,NaN,NaN,0.430,legor_uk,-1.431192,-1.088343,-1.006667,-1.975507,-1.700848,-1.245055
9345,Zimbabwe,2004.0,12019912.0,482.998411,47.3,0.406,0.441194,10.6,43.065,3.744,...,NaN,NaN,0.427,legor_uk,-1.553577,-1.207500,-1.000913,-2.063846,-1.801661,-1.333829
9553,Zimbabwe,2005.0,12076699.0,476.555324,47.2,0.407,0.441194,10.4,43.241,3.775,...,NaN,NaN,0.425,legor_uk,-1.674941,-1.272914,-1.331712,-2.236245,-1.818298,-1.309262
9761,Zimbabwe,2006.0,12155491.0,447.854924,47.2,0.408,0.441194,8.2,43.853,3.819,...,NaN,NaN,0.429,legor_uk,-1.539244,-0.932961,-1.225054,-1.959043,-1.740026,-1.358917
9968,Zimbabwe,2007.0,12255922.0,431.787188,47.1,0.409,0.441194,NaN,44.947,3.873,...,NaN,NaN,0.434,legor_uk,-1.571521,-1.108968,-1.258882,-2.156215,-1.798533,-1.401328
10175,Zimbabwe,2008.0,12379549.0,356.693350,47.0,0.410,0.441194,NaN,46.504,3.931,...,NaN,NaN,0.432,legor_uk,-1.549031,-1.213945,-1.528042,-2.140830,-1.781623,-1.355687


In [ ]:
df_final['Median Age']

In [350]:
# Drop all rows with with Year not in year_list
df_final = df_final[df_final['Year'].isin(year_list)]
df_final.shape    

(4352, 29)

# Principle Components Analysis

In [172]:
# Principle Components Analysis

def PCA_function(df, k):
  '''
  Function runs PCA analysis and prints results
  Argument 1: Data frame object
  Argument 2: Number of components to use in PCA
  Returns original dataset with its data transformed by PCA 
  '''

  # Drop rows with nan values; otherwise PCA function fails
  X = df.dropna()

  # Standardize the data
  scaler = StandardScaler()
  Y = scaler.fit_transform(X)

  # Creates instance of PCA with n_components as an argument
  pca = PCA(n_components=k)

  # Fit_transform on data
  Z = pca.fit_transform(Y)

  # Print results
  print("n_components:", k)
  print("Sum of Explained Variance Ratios:", sum(pca.explained_variance_ratio_))

  # Return label
  return Z

# Run PCA analysis with n=1...5 components
gov_columns = ['Voice and Accountability', 
              'Political Stability No Violence',
              'Government Effectiveness',
              'Regulatory Quality',
              'Rule of Law',
              'Control of Corruption']
for i in range(1, 7, 1):
  PCA_function(df_final[gov_columns], i)

n_components: 1
Sum of Explained Variance Ratios: 0.8474470692727674
n_components: 2
Sum of Explained Variance Ratios: 0.9157073052074033
n_components: 3
Sum of Explained Variance Ratios: 0.9640757423033324
n_components: 4
Sum of Explained Variance Ratios: 0.9844665440264843
n_components: 5
Sum of Explained Variance Ratios: 0.9925341085369412
n_components: 6
Sum of Explained Variance Ratios: 0.9999999999999999


In [184]:
PCA_function(df_final[gov_columns]., 1).shape

n_components: 1
Sum of Explained Variance Ratios: 0.847447069272768


(3973, 1)

In [186]:
df_final.shape

(4212, 29)

In [183]:
# Insert new column created with PCA and n_components=1
df_final['Quality of Government'] = PCA_function(df_final[gov_columns], 1)

# Drop original six gov columns
# df_final = df_final.drop(columns=gov_columns)

n_components: 1
Sum of Explained Variance Ratios: 0.8474470692727674


ValueError: Length of values does not match length of index

# Data Analysis

In [0]:
# Calculate the percentage of not-null values in each column

(df_final.notna().sum()/df_final.shape[0]).sort_values(ascending=False)

Country Name                       1.000000
Year                               1.000000
Distance from Equator              0.990458
Total Population                   0.988788
Urban Pop. per Capita              0.984017
Rule of Law                        0.978053
Voice and Accountability           0.974952
Political Stability No Violence    0.967319
Control of Corruption              0.964218
Government Effectiveness           0.960878
Regulatory Quality                 0.960878
GDP per Capita                     0.948473
Fertility Rate                     0.889074
Life Expectancy                    0.886927
Trade (% of GDP)                   0.877147
Inflation (CPI)                    0.830630
Gini Coefficient                   0.669609
Homicide Rate                      0.617605
Ethnic Fractionalization Index     0.560115
Gender Development Index           0.428197
Median Age                         0.184876
Human Capital Index                0.037452
National IQ Score               

In [0]:
# Maximum Likelihood
# https://towardsdatascience.com/all-about-missing-data-handling-b94b8b5d2184
  Describes my data as missing at random
# http://www.statisticalhorizons.com/wp-content/uploads/MissingDataByML.pdf
  Prefers ML too
# http://imaging.mrc-cbu.cam.ac.uk/statswiki/FAQ/emalgm?action=AttachFile&do=get&target=graham.pdf
  Prefers ML

In [0]:
# transform logarithm
# https://www.researchgate.net/post/log_transformation_and_standardization_which_should_come_first
#   Log first then standardize
# Issues: Maximum likelihood and multiple regression both assume sample is drawn from a multivariate normal distribution
# Need at aleast normal distribution
# https://www.ucl.ac.uk/child-health/short-courses-events/about-statistical-courses/statistics-and-research-methods/chapter-3-content/non
#   Method of Tukey's ladder of powers to transform data
# https://kenbenoit.net/assets/courses/ME104/logmodels2.pdf
# https://stats.stackexchange.com/questions/118373/what-are-the-advantages-of-using-log-gdp-per-capita-versus-simple-gdp-per-capita
# https://econbrowser.com/archives/2014/02/use-of-logarithms-in-economics

In [0]:
# MinMaxScaler 
# https://towardsdatascience.com/scale-standardize-or-normalize-with-scikit-learn-6ccc7d176a02
# gung - Reinstate Monica ... https://stats.stackexchange.com/questions/29781/when-conducting-multiple-regression-when-should-you-center-your-predictor-varia

In [0]:
# permutation importance
# https://www.kaggle.com/dansbecker/permutation-importance